# k30_sensor_test 

### - read -> write -> send to the database

In [ ]:
"""
Accessing the K30 10,000ppm CO2 sensor via I2C on Linux systems like Raspberry Pi.
USER: Change
            sensor_id = 'CO2-K30-01'
            output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv"
"""
try:
    from AtlasI2C import AtlasI2C
    import socket
    import sys
    import re
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    from threading import Timer
    from datetime import datetime, timezone
    import csv
    from collections import deque
    import queue
    from queue import Queue
    import traceback
    import io
    import fcntl
    import os
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

I2C_SLAVE = 0x0703
CMD_READ_REG = 0x22
REG_CO2_PPM = 0x08

class K30Sensor:
    def __init__(self, bus, addr=0x68):
        try:
            self.fr = io.open(bus, "rb", buffering=0)
            self.fw = io.open(bus, "wb", buffering=0)
            fcntl.ioctl(self.fr, I2C_SLAVE, addr)
            fcntl.ioctl(self.fw, I2C_SLAVE, addr)
        except Exception as e:
            logging.error(f"Failed to initialize K30Sensor: {e}")
            raise
        
    def __enter__(self):
        try:
            # Any additional setup if needed
            return self
        except Exception as e:
            logging.error(f"Error entering context: {e}")
            raise

    def read(self, count):
        s = self.fr.read(count)
        return list(s) if s else []

    def write(self, *data):
        if isinstance(data, (list, tuple)):
            data = bytes(data)
        self.fw.write(data)

    def read_co2_ppm(self):
        checksum = (CMD_READ_REG + REG_CO2_PPM) & 0xFF
        self.write(CMD_READ_REG, 0, REG_CO2_PPM, checksum)
        response = self.read(4)
        if len(response) == 4:
            return ((response[1] & 0xFF) << 8) | (response[2] & 0xFF)
        else:
            raise IOError("Invalid response from CO2 sensor")

    def close(self):
        self.fw.close()
        self.fr.close()
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            self.fr.close()
            self.fw.close()
        except Exception as e:
            logging.error(f"Error closing K30Sensor resources: {e}")

def connect_to_database():
    config = configparser.ConfigParser()
    config.read('config.ini')
    try:
        conn = psycopg2.connect(host=config['database']['host'],
                                dbname=config['database']['dbname'],
                                user=config['database']['user'],
                                password=config['database']['password'],
                                sslmode=config['database']['sslmode'])
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def write_header(output_file):
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(['Time Point', 'Device ID', 'Sensor ID', 'Reading Type', 'Reading Value', 'Error Code', 'Probe String'])

def write_reading(output_file, data_row):
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(data_row)

def main():
    conn = connect_to_database()
    if conn is None:
        sys.exit("Database connection failed")
    cursor = conn.cursor()

    output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv"

    try:
        with K30Sensor("/dev/i2c-1") as k30, open(output_filename, "w", buffering=1, newline='') as output_file:
            write_header(output_file)
            
            while True:
                errorMessage = ''  # Initialize errorMessage to an empty string
                probeString = ''  # Initialize probeString if not defined elsewhere
                reading_type = 'CO2'  # Define reading_type since it's used in your insert statement
                
                try:
                    co2_ppm = k30.read_co2_ppm()
                    if 0 < co2_ppm < 5000:
                        print(f"Concentration of CO2: {co2_ppm} ppm")
                        timestamp = datetime.now()
                        device_id = str(socket.gethostname())
                        sensor_id = 'CO2-K30-01'
                        reading_value = co2_ppm

                        # Write to CSV
                        data_row = [timestamp, device_id, sensor_id, reading_type, reading_value, errorMessage, probeString]
                        write_reading(output_file, data_row)
                        
                        # Insert into database
                        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
                        cursor.execute(header_db, data_row)
                        conn.commit()
                    else:
                        errorMessage = 'Invalid CO2 value range'
                        print("Received error in CO2 value, skipping...")
                except Exception as sensor_read_error:
                    errorMessage = str(sensor_read_error)
                    logging.error(f"Sensor read error: {sensor_read_error}")

                time.sleep(10)
    except Exception as e:
        logging.error(f"Error: {e}")
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    main()